In [2]:
import pandas as pd
from pymongo import MongoClient
import folium

In [3]:
locales = pd.read_csv("../output/requisitos.csv")
display(locales.head(),locales.shape)

,Unnamed: 0,category,name,location,latitude,longitude
0,0,airport,San Francisco International Airport,"{'type': 'Point', 'coordinates': [-122.3789554...",37.621313,-122.378955
1,0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.406831,...",37.789348,-122.406831
2,1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.4042684...",37.792814,-122.404268
3,2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.38783, ...",37.741198,-122.387830
4,3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.431519,...",37.782738,-122.431519


(106, 6)

In [4]:
locales.category.value_counts()

Starbucks     59
pub           40
vegan rest     6
airport        1
Name: category, dtype: int64

In [30]:
# coord SF [37.774929,-122.4194183]
m = folium.Map(
    location=[37.774929,-122.4194183],
    zoom_start=13,
    tiles='OpenStreetMap'
)
for i in range(len(locales.index)):
    dict_color ={"airport":"black","Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(locales.iloc[i][4]),float(locales.iloc[i][5])],
    popup=locales.iloc[i][1],
    icon=folium.Icon(color=dict_color[locales.iloc[i][1]])).add_to(m)

folium.Circle(
    radius=500,
    location=[37.790, -122.4209],
    color='black',
    fill=False,
).add_to(m)

folium.Circle(
    radius=1000,
    location=[37.7695, -122.4130],
    color='black',
    fill=False,
).add_to(m)


m

In [19]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [31]:
#compañias en el cículo [-122.4209,37.790]
cur = db.companiesSF.find({"office": {
         "$near": {
           "$geometry": {"type" : "Point" ,"coordinates":[-122.4209,37.790]},
           "$maxDistance": 600}}})

In [32]:
df = pd.DataFrame(cur)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,city,office,state,total_money_raised ($k),money_state
0,5e9d3c7155122cdd5bd9cf0a,Tivix,software,18.0,2008.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.423132,...",success,0.0,success2
1,5e9d3c7155122cdd5bd9ce6e,ICPA,consulting,30.0,1992.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.423065,...",success,0.0,success2
2,5e9d3c7155122cdd5bd9ce9a,Milobox,advertising,NaN,2007.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.421242,...",success,0.0,success2
3,5e9d3c7155122cdd5bd9cec9,Milobox,advertising,NaN,2007.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.421242,...",success,0.0,success2
4,5e9d3c7155122cdd5bd9ccf4,YourStreet,web,6.0,2006.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.421341,...",success,250.0,success


(6, 10)

In [33]:
#locales en el cículo [-122.4209,37.790]
cur = db.locales.find({"location": {
         "$near": {
           "$geometry": {"type" : "Point" ,"coordinates":[-122.4209,37.790]},
           "$maxDistance": 500}}})

In [34]:
df_locales_circulo=pd.DataFrame(cur)

In [35]:
df_locales_circulo.head()

,_id,category,name,location,latitude,longitude
0,5e9d7a8255122cdd5bd9de13,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.422065,...",37.790218,-122.422065
1,5e9d7a8255122cdd5bd9de5b,vegan rest,Vegan Picnic,"{'type': 'Point', 'coordinates': [-122.4205203...",37.788996,-122.420520
2,5e9d7a8255122cdd5bd9de4c,pub,Lush Lounge,"{'type': 'Point', 'coordinates': [-122.4204128...",37.788126,-122.420413


In [36]:
# coord cículo [37.790,-122.4209]
m = folium.Map(
    location=[37.790,-122.4209],
    zoom_start=15,
    tiles='OpenStreetMap'
)
for i in range(len(df_locales_circulo.index)):
    dict_color ={"Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(df_locales_circulo.iloc[i][4]),float(df_locales_circulo.iloc[i][5])],
    popup=df_locales_circulo.iloc[i][1],
    icon=folium.Icon(color=dict_color[df_locales_circulo.iloc[i][1]])).add_to(m)

for i in range(len(df.index)):
    folium.Marker(
    location=[float(df.iloc[i][6]["coordinates"][1]),float(df.iloc[i][6]["coordinates"][0])],
    popup=df.iloc[i][1],
    icon=folium.Icon(color="black")).add_to(m)

folium.Circle(
    radius=500,
    location=[37.790,-122.4209],
    color='black',
    fill=False,
).add_to(m)


m